In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import open3d as o3d
import os

import open3d as o3d
import numpy as np
import PIL.Image
import IPython.display
import os
import urllib.request
import tarfile
import gzip
import zipfile
import shutil

interactive = True
def filter_points(df, k=50, threshold=0.5):

    # Nearest neighbours outlier setup
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(df)
    neighbors_and_distances = knn.kneighbors(df)
    knn_distances = neighbors_and_distances[0]
    neighbors = neighbors_and_distances[1]
    kth_distance = [x[-1] for x in knn_distances]
    tnn_distance = np.mean(knn_distances, axis=1)


    return df[abs(tnn_distance - np.mean(tnn_distance)) < threshold * np.std(tnn_distance)]

def rotate_scan(target, source, voxel_size=0.05):

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    result_fast = execute_fast_global_registration(source_down, target_down,
                                               source_fpfh, target_fpfh,
                                               voxel_size)

    print(result_fast)
    print(result_fast.transformation)
    
    return source.transform(result_fast.transformation)

def preprocess_point_cloud(pcd, voxel_size):

    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 5
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 10
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    
    return pcd_down, pcd_fpfh

def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):

    distance_threshold = voxel_size * 2
    print(":: Apply fast global registration with distance threshold %.3f" \
            % distance_threshold)
    result = o3d.registration.registration_fast_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))

    return result

def convert_to_o3d(point_list):
    
    # Convert points to open 3D point cloud    
    xyz = np.array(point_list)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)

    return pcd


## From http://www.open3d.org/docs/release/tutorial/Advanced/multiway_registration.html
def pairwise_registration(source, target):
    print("Apply point-to-plane ICP")
    icp_coarse = o3d.registration.registration_icp(
        source, target, max_correspondence_distance_coarse, np.identity(4),
        o3d.registration.TransformationEstimationPointToPlane())
    icp_fine = o3d.registration.registration_icp(
        source, target, max_correspondence_distance_fine,
        icp_coarse.transformation,
        o3d.registration.TransformationEstimationPointToPlane())
    transformation_icp = icp_fine.transformation
    information_icp = o3d.registration.get_information_matrix_from_point_clouds(
        source, target, max_correspondence_distance_fine,
        icp_fine.transformation)
    return transformation_icp, information_icp


def full_registration(pcds, max_correspondence_distance_coarse,
                      max_correspondence_distance_fine):
    pose_graph = o3d.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id])
            print("Build o3d.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.registration.PoseGraphNode(np.linalg.inv(odometry)))
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                   target_id,
                                                   transformation_icp,
                                                   information_icp,
                                                   uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                   target_id,
                                                   transformation_icp,
                                                   information_icp,
                                                   uncertain=True))
    return pose_graph

pcds = []
filenames = []
voxel_size = 0.005
allowed_point_threshold = 500
for i in range(10):
    filenames.append('rawlidar0scan{}.xyz'.format(i))

for filename in filenames:
    # Import points and convert to open3d class
    points = pd.read_csv(filename, header=None).rename(columns={0:'x', 1:'y', 2:'z'})
    pointsFiltered = convert_to_o3d(filter_points(points, k=10, threshold=0.3))
    
    # Downsample to specific voxel size
    pointsDownSampled = pointsFiltered.voxel_down_sample(voxel_size=voxel_size)
    
#     if len(pointsDownSampled.points) < allowed_point_threshold:
#         print(len(pointsDownSampled.points), "less than threshold({})".format(allowed_point_threshold))
#         pass
#     else:

# Compute normals
    pointsDownSampled.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5, max_nn=30))
    pointsDownSampled.orient_normals_to_align_with_direction()
    # Add to list
    pcds.append(pointsDownSampled)

pcds_down = pcds
pcds

[geometry::PointCloud with 4178 points.,
 geometry::PointCloud with 24030 points.,
 geometry::PointCloud with 1350 points.,
 geometry::PointCloud with 517 points.,
 geometry::PointCloud with 19516 points.,
 geometry::PointCloud with 4241 points.,
 geometry::PointCloud with 6145 points.,
 geometry::PointCloud with 23042 points.,
 geometry::PointCloud with 561 points.,
 geometry::PointCloud with 17 points.]

In [2]:
max_correspondence_distance_coarse = voxel_size * 15
max_correspondence_distance_fine = voxel_size * 1.5

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    pose_graph = full_registration(pcds_down,
                                   max_correspondence_distance_coarse,
                                   max_correspondence_distance_fine)
    
    
converg_criteria = o3d.registration.GlobalOptimizationConvergenceCriteria()
# converg_criteria.min_residual = 6.57e-08
converg_criteria.max_iteration = 5000


print("Optimizing PoseGraph ...")
option = o3d.registration.GlobalOptimizationOption(
    max_correspondence_distance=max_correspondence_distance_fine,
    edge_prune_threshold=0.25,
    reference_node=0)
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    o3d.registration.global_optimization(
        pose_graph, o3d.registration.GlobalOptimizationLevenbergMarquardt(), converg_criteria, option)
    
    
# Write to file
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(pcds)):
    pcds[point_id].transform(pose_graph.nodes[point_id].pose)
    pcd_combined += pcds[point_id]
pcd_combined_down = pcd_combined.voxel_down_sample(voxel_size=voxel_size)
o3d.io.write_point_cloud("multiway_registration.xyz", pcd_combined_down)

Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
[Open3D DEBUG]

[Open3D DEBUG] ICP Iteration #24: Fitness 0.7548, RMSE 0.0154
[Open3D DEBUG] Residual : 9.85e-05 (# of elements : 18138)
[Open3D DEBUG] ICP Iteration #25: Fitness 0.7548, RMSE 0.0154
[Open3D DEBUG] Residual : 9.85e-05 (# of elements : 18138)
[Open3D DEBUG] ICP Iteration #26: Fitness 0.7548, RMSE 0.0154
[Open3D DEBUG] Residual : 9.85e-05 (# of elements : 18138)
[Open3D DEBUG] ICP Iteration #27: Fitness 0.7548, RMSE 0.0154
[Open3D DEBUG] Residual : 9.85e-05 (# of elements : 18138)
[Open3D DEBUG] ICP Iteration #0: Fitness 0.4838, RMSE 0.0040
[Open3D DEBUG] Residual : 6.98e-06 (# of elements : 11625)
[Open3D DEBUG] ICP Iteration #1: Fitness 0.5831, RMSE 0.0037
[Open3D DEBUG] Residual : 1.20e-06 (# of elements : 14012)
[Open3D DEBUG] ICP Iteration #2: Fitness 0.5822, RMSE 0.0037
[Open3D DEBUG] Residual : 1.03e-06 (# of elements : 13991)
[Open3D DEBUG] ICP Iteration #3: Fitness 0.5828, RMSE 0.0037
[Open3D DEBUG] Residual : 1.05e-06 (# of elements : 14004)
[Open3D DEBUG] ICP Iteration #4: Fit

[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0039, RMSE 0.0280
[Open3D DEBUG] Residual : 6.25e-04 (# of elements : 2)
[Open3D DEBUG] ICP Iteration #1: Fitness 0.0542, RMSE 0.0166
[Open3D DEBUG] Residual : 2.49e-04 (# of elements : 28)
[Open3D DEBUG] ICP Iteration #2: Fitness 0.0039, RMSE 0.0482
[Open3D DEBUG] Residual : 2.30e-03 (# of elements : 2)
[Open3D DEBUG] ICP Iteration #3: Fitness 0.0000, RMSE 0.0000
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, R

[Open3D DEBUG] ICP Iteration #0: Fitness 0.0381, RMSE 0.0056
[Open3D DEBUG] Residual : 1.38e-05 (# of elements : 234)
[Open3D DEBUG] ICP Iteration #1: Fitness 0.0018, RMSE 0.0060
[Open3D DEBUG] Residual : 6.60e-06 (# of elements : 11)
[Open3D DEBUG] ICP Iteration #2: Fitness 0.0000, RMSE 0.0000
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0208, RMSE 0.0532
[Open3D DEBUG] Residual : 9.41e-04 (# of elements : 128)
[Open3D DEBUG] ICP Iteration #1: Fitness 0.0203, RMSE 0.0527
[Open3D DEBUG] Residual : 8.09e-04 (# of elements : 125)
[Open3D DEBUG] ICP Iteration #2: Fitness 0.0194, RMSE 0.0536
[Open3D DEBUG] Residual : 7.05e-04 (# of elements : 119)
[Open3D DEBUG] ICP Iteration #3: Fitness 0.0177, RMSE 0.0545
[Open3D DEBUG] Residual : 6.56e-04 (# of elements :

True

In [8]:
converg_criteria

GlobalOptimizationConvergenceCriteria
> max_iteration : 100
> min_relative_increment : 0.000001
> min_relative_residual_increment : 0.000001
> min_right_term : 0.000001
> min_residual : 0.050000
> max_iteration_lm : 20
> upper_scale_factor : 0.666667
> lower_scale_factor : 0.333333

In [26]:
o3d.registration.GlobalOptimizationConvergenceCriteria()

GlobalOptimizationConvergenceCriteria
> max_iteration : 100
> min_relative_increment : 0.000001
> min_relative_residual_increment : 0.000001
> min_right_term : 0.000001
> min_residual : 0.000001
> max_iteration_lm : 20
> upper_scale_factor : 0.666667
> lower_scale_factor : 0.333333